<a href="https://colab.research.google.com/github/jia-xiaof/hello-word/blob/master/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Movie recommendation

# 1M数据集也从官网wget

In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2019-01-06 08:29:05--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  4.13MB/s    in 1.4s    

2019-01-06 08:29:07 (4.13 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



In [0]:
!unzip ml-1m.zip

# 10M数据集直接从官网wget,很快

In [0]:
!wget http://files.grouplens.org/datasets/movielens/ml-10m.zip

In [0]:
!unzip ml-10m.zip

In [0]:
ll

In [0]:
ll ml-10M100K/


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

header = ['user_id', 'movie_id', 'rating', 'timestamp']
#dataset = pd.read_csv('u.data', sep='\t',  engine='python')  # ML-100K数据集
dataset = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python',names=header)  # ML-1M数据集
#dataset = pd.read_csv('ml-10M100K/ratings.dat', sep='::', engine='python',names=header)  # ML-10M数据集
print(dataset.head())
train, valbase = train_test_split(dataset, test_size=0.2, random_state=42)
val, test = train_test_split(valbase, test_size=0.5, random_state=42)

n_users = len(dataset.user_id.unique())
n_movies = len(dataset.movie_id.unique())

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Dropout
from keras.models import Model
from keras import regularizers, optimizers
from keras import backend as K
from keras.callbacks import TensorBoard

k = 128
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_movies + 1, k, name="Movie-Embedding")(movie_input)
movie_vec0 = Flatten(name="Flatten-Movies")(movie_embedding)
movie_vec = Dense(k,  kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation='relu')(movie_vec0)
#movie_vec = Dropout(0.2)(movie_vec1)


user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, k, name="User-Embedding")(user_input)
user_vec0 = Flatten(name="Flatten-Users")(user_embedding)
user_vec = Dense(k,  kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation='relu')(user_vec0)
#user_vec = Dropout(0.2)(user_vec1)

x = Dot(name="Dot-Product", axes=1)([movie_vec, user_vec])
#x = Dense(64, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation='relu')(x)
#x = Dense(64, activation='relu')(x)
#x = Dropout(0.2)(x)
#prod = Dense(1, activation='linear')(x)

def mymse(y_true, y_pred):
    loss1 = K.mean((y_true-y_pred)**2, -1)
    loss2 = 0.0001 * K.mean(user_vec ** 2 + movie_vec ** 2, -1)
    return loss1 + loss2

model = Model([user_input, movie_input], x)
# model.compile('adam', 'mean_squared_error')
#model.compile(optimizer=optimizers.adam(0.001), loss='mean_squared_error')
model.compile(optimizer=optimizers.adam(0.001), loss=mymse)

model.fit([train.user_id, train.movie_id], train.rating,
          validation_data=([val.user_id, val.movie_id], val.rating),
          #callbacks=[TensorBoard(log_dir='./movie_log')], #用TensorBoard可视化训练过程
          epochs=600, batch_size=3000)
model.save('mymovie-recommend.h5')


In [7]:
!git clone https://github.com/wxs/keras-mnist-tutorial.git 'TerenceDrive/Colab Notebooks/keras-mnist-tutorial'

Cloning into 'TerenceDrive/Colab Notebooks/keras-mnist-tutorial'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.


In [0]:
显示硬件环境

In [0]:
from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

显示内存信息

In [2]:
!cat /proc/meminfo 

cat: /proc/meminfo : No such file or directory


显示CPU信息

In [0]:
!cat /proc/cpuinfo

显示版本

In [4]:
!cat /proc/version

Linux version 4.14.79+ (chrome-bot@swarm-cros-634) (gcc version 4.9.x 20150123 (prerelease) (4.9.2_cos_gg_4.9.2-r193-ac6128e0a17a52f011797f33ac3e7d6273a9368d_4.9.2-r193)) #1 SMP Wed Dec 19 21:19:13 PST 2018


显示设备：

In [0]:
!cat /proc/devices

显示空间

In [0]:
!df

In [0]:
!tensorboard --logdir='./log'

In [0]:
from tensorflow.python.keras.models import load_model
model = load_model('mymovie-recommend.h5')
#model.evaluate([test.user_id, test.book_id], test.rating, batch_size=10000)
#model.fit([train.user_id, train.book_id], train.rating, validation_data=([test.user_id, test.book_id], test.rating), epochs=500, initial_epoch=200, batch_size=3000)
model.fit([train.user_id, train.movie_id], train.rating,
          validation_data=([val.user_id, val.movie_id], val.rating),
          #callbacks=[TensorBoard(log_dir='./movie_log')], #用TensorBoard可视化训练过程
          epochs=500, initial_epoch=200, batch_size=3000) 
model.save('mymovie-recommend.h5')